# Working with OpenINTEL data in pandas/pyarrow

Note that pandas is sufficient for smaller computation loads. OpenINTEL data can be large, and pandas might not be suitable.
Other tools that we have available at DACS include a Spark based compute cluster, and clickhouseDB.

In [1]:
import pandas as pd
import requests
import io

# You also need to have the package 'pyarrow' installed!

In [4]:
# OpenINTEL open data URL, full directory available at: https://openintel.nl/download/
openintel_url = 'https://object.openintel.nl/openintel-public/fdns/basis=toplist/source=tranco/year=2024/month=01/day=01/part-00000-c96c4d48-1931-4519-8c15-95d0ddaa128a-c000-92c637e9-2f09-47fc-8379-d76fc284e888.gz.parquet'

In [5]:
# Download the file into memory
response = requests.get(openintel_url)
response.raise_for_status()  # Raise an error if the download fails

In [6]:
# Read the parquet file into a DataFrame
df = pd.read_parquet(io.BytesIO(response.content), engine="pyarrow")

# Display the first few rows
print(df.head())

  query_type                                   query_name response_type  \
0          A  xn--12cs4acv8cd1d0ae5bg9g1jsac.xn--t60b56a.             A   
1          A  xn--12cs4acv8cd1d0ae5bg9g1jsac.xn--t60b56a.             A   
2       AAAA  xn--12cs4acv8cd1d0ae5bg9g1jsac.xn--t60b56a.          AAAA   
3       AAAA  xn--12cs4acv8cd1d0ae5bg9g1jsac.xn--t60b56a.          AAAA   
4         NS  xn--12cs4acv8cd1d0ae5bg9g1jsac.xn--t60b56a.            NS   

                                 response_name  response_ttl      timestamp  \
0  xn--12cs4acv8cd1d0ae5bg9g1jsac.xn--t60b56a.         300.0  1704074589000   
1  xn--12cs4acv8cd1d0ae5bg9g1jsac.xn--t60b56a.         300.0  1704074589000   
2  xn--12cs4acv8cd1d0ae5bg9g1jsac.xn--t60b56a.         300.0  1704074589000   
3  xn--12cs4acv8cd1d0ae5bg9g1jsac.xn--t60b56a.         300.0  1704074589000   
4  xn--12cs4acv8cd1d0ae5bg9g1jsac.xn--t60b56a.       86400.0  1704074589000   

        rtt   worker_id  status_code  ad_flag  ... cdnskey_pk_eddsa_a  \
0

In [5]:
df.columns

Index(['query_type', 'query_name', 'response_type', 'response_name',
       'response_ttl', 'timestamp', 'rtt', 'worker_id', 'status_code',
       'ad_flag', 'section', 'ip4_address', 'ip6_address', 'country', 'as',
       'as_full', 'ip_prefix', 'cname_name', 'dname_name', 'mx_address',
       'mx_preference', 'ns_address', 'txt_text', 'ds_key_tag', 'ds_algorithm',
       'ds_digest_type', 'ds_digest', 'dnskey_flags', 'dnskey_protocol',
       'dnskey_algorithm', 'dnskey_pk_rsa_n', 'dnskey_pk_rsa_e',
       'dnskey_pk_rsa_bitsize', 'dnskey_pk_eccgost_x', 'dnskey_pk_eccgost_y',
       'dnskey_pk_dsa_t', 'dnskey_pk_dsa_q', 'dnskey_pk_dsa_p',
       'dnskey_pk_dsa_g', 'dnskey_pk_dsa_y', 'dnskey_pk_eddsa_a',
       'dnskey_pk_wire', 'nsec_next_domain_name', 'nsec_owner_rrset_types',
       'nsec3_hash_algorithm', 'nsec3_flags', 'nsec3_iterations', 'nsec3_salt',
       'nsec3_next_domain_name_hash', 'nsec3_owner_rrset_types',
       'nsec3param_hash_algorithm', 'nsec3param_flags',
       '

In [31]:
tranco_a_df = df[df['response_type'] == 'CNAME'] # Filter on DNS response type A
# tranco_a_ip4_df = tranco_a_df[tranco_a_df['ip4_address'] == "170.114.52.2"]
# tranco_a_ip4_df[['ip4_address', 'query_name', 'response_name', 'cname_name']]
df2 = tranco_a_df[tranco_a_df["response_name"].str.contains('ultradns.net', case=False, na=False)]
df2
# tranco_a_df["response_name"].unique()[0:10]

,query_type,query_name,response_type,response_name,response_ttl,timestamp,rtt,worker_id,status_code,ad_flag,...,cdnskey_pk_eddsa_a,cdnskey_pk_wire,caa_flags,caa_tag,caa_value,tlsa_usage,tlsa_selector,tlsa_matchtype,tlsa_certdata,ptr_name
3990014,TXT,_dmarc.ultradns.net.,CNAME,_dmarc.ultradns.net.,300.0,1704071178000,0.030493,1582299107,0,0,...,None,None,NaN,None,None,NaN,NaN,NaN,None,None


In [16]:
tranco_a_df = df[df['response_type'] == 'A'] # Filter on DNS response type A
tranco_a_ip4_df = tranco_a_df[tranco_a_df['ip4_address'] == "170.114.52.2"]
tranco_a_ip4_df[['ip4_address', 'query_name', 'response_name']]

,ip4_address,query_name,response_name
3925041,170.114.52.2,zoom.us.,zoom.us.
3925096,170.114.52.2,www.zoom.us.,zoom.us.


In [6]:
# Process the OpenINTEL data
tranco_a_df = df[df['response_type'] == 'A'] # Filter on DNS response type A
tranco_a_ip4_df = tranco_a_df['ip4_address']  # Get the column we want

tranco_a_ip4_nparray = tranco_a_ip4_df.unique()  # Only grab unique values

tranco_a_df['domain'] = tranco_a_df['response_name'].str.rstrip('.')

tranco_domain_a_df = tranco_a_df[['domain', 'ip4_address']].copy()
unique_tranco_domain_a_df = tranco_domain_a_df.drop_duplicates()
 # Sometimes duplications are found.
# Print count
print('Total IP4 addresses found: %s and unique addresses are %s' %((len(tranco_a_ip4_df), len(tranco_a_ip4_nparray))))

/tmp/ipykernel_66333/878748865.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tranco_a_df['domain'] = tranco_a_df['response_name'].str.rstrip('.')


Total IP4 addresses found: 2915481 and unique addresses are 598212


In [7]:
unique_tranco_domain_a_df.to_csv("../data/tranco-1m/openintel_domain_a_record.csv", index=False)

In [8]:
# Save ranks of tranco domains in a csv file: tranco_domains_with_rank.csv
import csv

# Load domains and assign ranks
rank_dict = {}
with open("../data/tranco-1m/top-1m.txt", "r", encoding="utf-8") as f, open("../data/tranco-1m/tranco_domains_with_rank.csv", "w") as out:
    writer = csv.writer(out)
    writer.writerow(["domain", "rank"])
    
    for rank, domain in enumerate(f, start=1):  # Auto-assign rank
        domain = domain.strip()
        rank_dict[domain] = str(rank)
        writer.writerow([domain, rank])

In [9]:
# Find ranks of tranco 1M domains with their A records (IP address/es) and save it into a file.
import csv
import pandas as pd

# 1. Read tranco_domain_a_df that has domain name and ranks: openintel_domain_a_record
# 2. Join it with tranco_domains_with_rank
df1 = pd.read_csv("../data/tranco-1m/tranco_domains_with_rank.csv")
df2 = pd.read_csv("../data/tranco-1m/openintel_domain_a_record.csv")
df3 = df1.merge(df2, how='left', on='domain')

df3.to_csv("../data/tranco-1m/openintel_tranco_resolved_with_rank.csv", index=False)

In [10]:
df2[df2["domain"] == "zoom.us"]

,domain,ip4_address
739081,zoom.us,170.114.52.2


In [11]:
df3[df3["domain"] == "zoom.us"]

,domain,rank,ip4_address
131,zoom.us,76,170.114.52.2


In [18]:
# Find the number of matched IP addresses 

# Compare the ip addresses with the protected prefixes using pytricia loop
import pandas as pd
import ipaddress
import pytricia


def build_prefix_trees(prefix_list):
    """Builds two Patricia Tries: one for IPv4 and one for IPv6."""
    pt_v4 = pytricia.PyTricia()
    pt_v6 = pytricia.PyTricia()

    for prefix in prefix_list:
        network = ipaddress.ip_network(prefix)
        if network.version == 4:

            pt_v4[prefix] = True
        else:
            pt_v6[prefix] = True

    return pt_v4, pt_v6

def count_covered_ips(ip_list, pt_v4, pt_v6):
    """Counts how many IPs are covered by the prefixes in the Patricia Tries."""
    count = 0
    for ip in ip_list:
        ip_obj = ipaddress.ip_address(ip)
        if ip_obj.version == 4 and pt_v4.get(ip):
            count += 1
        elif ip_obj.version == 6 and pt_v6.get(ip):
            count += 1

    return count


def get_covered_ips(ip_list, pt_v4, pt_v6):
    """Returns the list of IPs covered by prefixes in the Patricia Trie."""
    covered_ips = []
    for ip in ip_list:
        ip_obj = ipaddress.ip_address(ip)
        if ip_obj.version == 4 and pt_v4.get(ip):
            covered_ips.append(ip)
        elif ip_obj.version == 6 and pt_v6.get(ip):
            covered_ips.append(ip)
#     covered_ips = [ip for ip in ip_list if prefix_tree.get(ip)]
    return covered_ips



# Protected prefixes lists using all five scrubberscovered_count
df = pd.read_csv("../data/customers_prefixes_scrubber_all_2024.csv") 
protected_prefixes = df["prefix"].tolist()

# Openintel list of IP addresses
df = pd.read_csv("../data/tranco-1m/openintel_tranco_resolved_with_rank.csv") 
df_cleaned = df.dropna()

tranco_ip_addresses = df_cleaned["ip4_address"].unique()

# Build separate prefix trees for IPv4 and IPv6
pt_v4, pt_v6 = build_prefix_trees(protected_prefixes)

# Find the number of IPs covered
covered_count = count_covered_ips(tranco_ip_addresses, pt_v4, pt_v6)

# Find the IPs covered
covered_ips = get_covered_ips(tranco_ip_addresses, pt_v4, pt_v6)


# Save covered ips in a .txt file.
with open("../data/tranco-1m/openintel_tranco_scrubber_covered_ip.txt", "w", encoding="utf-8") as f:
    for ip in covered_ips:
        f.write(f"{ip}\n")
print(f"Number of IPs protected by the five scrubber's protected prefixes is: {len(covered_ips)} and IPs saved in openintel_tranco_scrubber_covered_ip.txt")


Number of IPs protected by the five scrubber's protected prefixes is: 6794 and IPs saved in openintel_tranco_scrubber_covered_ip.txt


In [24]:
# Find ranks of tranco 1M domains with their A records (IP address/es) and save it into a file.
import csv

covered_ips = []
# Parse massdns results
with open("../data/tranco-1m/openintel_tranco_scrubber_covered_ip.txt", "r", encoding="utf-8") as f:
    for line in f:
        ip = line.strip("\n")
        covered_ips.append(ip)
        
# Convert it into a dataframe with column name ipv4_address
df1 = pd.DataFrame(covered_ips, columns=['ip4_address'])

df2 = pd.read_csv("../data/tranco-1m/openintel_tranco_resolved_with_rank.csv")

# df3 = df2.merge(df1, how='inner', on='ip4_address')
        
    
result = df1.merge(df2, on='ip4_address', how='inner')
result.to_csv("../data/tranco-1m/openintel_ip_domains_ranks.csv", index=False)

print("%s number of domains are protected. \n" %len(result))


31648 number of domains are protected. 



In [52]:
# Find the number of domains hosted by those IP addresses
# Reverse IP lookup, results saved in /home/shyam/jupy/ddos_scrubber/data/tranco-1m/ttranco_scrubber_covered_ptr_results.txt
import pandas as pd
from ipaddress import ip_address

# Input and output file names
df = pd.read_csv("../data/tranco-1m/openintel_ip_domains_ranks.csv")
ptr_file = "../data/tranco-1m/openintel_tranco_scrubber_covered_ptr.txt"

df_new = df.copy()
def reverse_ptr(ip):
    return ip_address(ip).reverse_pointer

df_new['rev_ptr'] = df_new['ip4_address'].apply(reverse_ptr)

df_new.to_csv("../data/tranco-1m/openintel_tranco_scrubber_covered_ptr.csv")


In [59]:
import pandas as pd
ptr = pd.read_csv("../data/tranco-1m/openintel_tranco_scrubber_covered_ptr.csv")
a = ptr["rev_ptr"].unique()

# Save covered ips in a .txt file.
with open("../data/tranco-1m/openintel_tranco_scrubber_covered_ptr_only.txt", "w", encoding="utf-8") as f:
    for ip in a:
        f.write(f"{ip}\n")
print(f"Number of IPs protected by the five scrubber's protected prefixes is: {len(a)} and IPs saved in openintel_tranco_scrubber_covered_ptr_only.txt")

Number of IPs protected by the five scrubber's protected prefixes is: 6794 and IPs saved in openintel_tranco_scrubber_covered_ptr_only.txt


In [51]:
df_new

,ip4_address,domain,rank,rev_ptr
0,170.114.52.2,zoom.us,76,2.52.114.170.in-addr.arpa
1,141.193.213.20,applovin.com,240,20.213.193.141.in-addr.arpa
2,141.193.213.20,forter.com,591,20.213.193.141.in-addr.arpa
3,141.193.213.20,berkeley.edu,636,20.213.193.141.in-addr.arpa
4,141.193.213.20,conviva.com,752,20.213.193.141.in-addr.arpa
...,...,...,...,...
31643,63.250.43.139,aktbaraty.com,998971,139.43.250.63.in-addr.arpa
31644,109.234.165.79,acces-sap.fr,999184,79.165.234.109.in-addr.arpa
31645,198.54.125.51,cinehindi.com,999238,51.125.54.198.in-addr.arpa
31646,160.153.0.40,kfapfakes.com,999316,40.0.153.160.in-addr.arpa


In [47]:
df_new

,ip4_address,domain,rank,rev_ptr
0,170.114.52.2,zoom.us,76,76.0.0.0.in-addr.arpa
1,141.193.213.20,applovin.com,240,240.0.0.0.in-addr.arpa
2,141.193.213.20,forter.com,591,79.2.0.0.in-addr.arpa
3,141.193.213.20,berkeley.edu,636,124.2.0.0.in-addr.arpa
4,141.193.213.20,conviva.com,752,240.2.0.0.in-addr.arpa
...,...,...,...,...
31643,63.250.43.139,aktbaraty.com,998971,59.62.15.0.in-addr.arpa
31644,109.234.165.79,acces-sap.fr,999184,16.63.15.0.in-addr.arpa
31645,198.54.125.51,cinehindi.com,999238,70.63.15.0.in-addr.arpa
31646,160.153.0.40,kfapfakes.com,999316,148.63.15.0.in-addr.arpa


In [38]:
df.columns

Index(['ip4_address', 'domain', 'rank'], dtype='object')